In [1]:
import os
import random

import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 200 #make plots bigger

import cv2
import xml.etree.ElementTree as ET

PROCESSING_DIM = 256 #divisible by 32 for u-net

#from google.colab import drive
#drive.mount('/content/gdrive')
#PATH = './gdrive/My Drive/goban_data_set/'
PATH = './'

In [2]:
def mirror_half(img, y):
    if random.random() < 0.5:
        return img, y
    else:
        img_mirrored = img[:,::-1].copy() # y,x,c
        y_mirrored = y.copy()
        # walk through x coordinates and mirror
        for i in range(0, y.shape[0], 2):
            y_mirrored[i] = 999 - y[i] #based on coordinates for 1000x1000
        # swap left and right corners
            y_mirrored_copy = y_mirrored.copy()
            y_mirrored[0: 2] = y_mirrored_copy[6: 8]
            y_mirrored[2: 4] = y_mirrored_copy[4: 6]
            y_mirrored[4: 6] = y_mirrored_copy[2: 4]
            y_mirrored[6: 8] = y_mirrored_copy[0: 2]
        return img_mirrored, y_mirrored
        
def random_cutout(img, labels, max_cutout=50):
    #print(f"labels: {labels}")
    labels = labels.astype('float32')
    left_space = np.amin(labels[::2]) - 1
    right_space = 999 - np.amax(labels[::2])
    top_space = np.amin(labels[1::2]) - 1
    bottom_space = 999 - np.amax(labels[1::2])
    
    safe_space = min(left_space, right_space, top_space, bottom_space)
    safe_space = min(safe_space, max_cutout) #TODO: change other versions of notebook to min as well

    rand_left = random.randint(0, safe_space)
    rand_right = random.randint(0, safe_space)
    horizontal_total = rand_left + rand_right
    rand_top = random.randint(0, safe_space)
    rand_bottom = horizontal_total - rand_top #keeps aspect ratio
    
    cutoff_left = rand_left
    cutoff_top = rand_top
    cutoff_right = 999 - rand_right
    cutoff_bottom = 999 - rand_bottom
    
    cutout = img[cutoff_top:cutoff_bottom, cutoff_left:cutoff_right]
    
    for xcoord in range(0, len(labels), 2):
        labels[xcoord] =  labels[xcoord] * (1000 / (1000 - rand_right))
        
        labels[xcoord] = 999 - labels[xcoord]
        labels[xcoord] =  labels[xcoord] * (1000 / (1000 - rand_left))
        labels[xcoord] = 999 - labels[xcoord]
        
    for ycoord in range(1, len(labels), 2):
        labels[ycoord] = labels[ycoord] * (1000 / (1000 - rand_bottom))
        
        labels[ycoord] = 999 - labels[ycoord]
        labels[ycoord] =  labels[ycoord] * (1000 / (1000 - rand_top))
        labels[ycoord] = 999 - labels[ycoord]
    
    labels = np.rint(labels)
    labels = labels.astype('uint16')
    return cutout, labels

def create_training_images_and_masks(m, shift=0):
    files = os.listdir(PATH + 'training_images_for_labels/')
    for i in range(0+shift, m+shift):
        for file in random.sample(files, 1):
            y = np.zeros((8), dtype=np.uint16)
            points = {}
            tree = ET.parse(PATH+'training_corner_labels/'+file.rstrip('png')+'xml')
            root = tree.getroot()
            for obj in root.findall('object'):
                tag = obj.find('name').text
                x_coord = np.uint16(obj.find('point').find('x1').text)
                y_coord = np.uint16(obj.find('point').find('y1').text)
                points[tag] = [x_coord,y_coord]
            y[0:2] = points['ul_board_corner']
            y[2:4] = points['dl_board_corner']
            y[4:6] = points['dr_board_corner']
            y[6:8] = points['ur_board_corner']
            
            filename=PATH+'training_images_for_labels/'+file
            img = cv2.imread(filename)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            img = clahe.apply(img)
            img, y = random_cutout(img, y)
            img = cv2.resize(img, dsize=(PROCESSING_DIM,PROCESSING_DIM), 
                             interpolation=cv2.INTER_LINEAR)
            mirror_half(img, y)
            filename=PATH+'training_images_for_masks/'+str(i)+'.png'
            cv2.imwrite(filename, img)
            
            mask = np.zeros((1000, 1000), np.uint8)
            cv2.fillConvexPoly(mask, np.reshape(y, (-1,2)).astype('int32'), 255)
            mask = cv2.resize(mask, dsize=(PROCESSING_DIM,PROCESSING_DIM), 
                         interpolation=cv2.INTER_LINEAR)
            filename=PATH+'training_masks/'+str(i)+'.png'
            cv2.imwrite(filename, mask)
    print('created randomized training images and masks')
    return None
        
def create_validation_images_and_masks():
    files = os.listdir(PATH + 'validation_images/')
    for file in files:
        y = np.zeros((8), dtype=np.uint16)
        points = {}
        tree = ET.parse(PATH+'validation_labels/'+file.rstrip('png')+'xml')
        root = tree.getroot()
        for obj in root.findall('object'):
            tag = obj.find('name').text
            x_coord = np.uint16(obj.find('point').find('x1').text)
            y_coord = np.uint16(obj.find('point').find('y1').text)
            points[tag] = [x_coord,y_coord]
        y[0:2] = points['ul_board_corner']
        y[2:4] = points['dl_board_corner']
        y[4:6] = points['dr_board_corner']
        y[6:8] = points['ur_board_corner']
        
        filename=PATH+'validation_images/'+file
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        img = clahe.apply(img)
        img = cv2.resize(img, dsize=(PROCESSING_DIM,PROCESSING_DIM), 
                         interpolation=cv2.INTER_LINEAR)
        filename=PATH+'validation_images_for_masks/'+file
        cv2.imwrite(filename, img)
        
        mask = np.zeros((1000, 1000), np.uint8)
        cv2.fillConvexPoly(mask, np.reshape(y, (-1,2)).astype('int32'), 255)
        mask = cv2.resize(mask, dsize=(PROCESSING_DIM,PROCESSING_DIM), 
                         interpolation=cv2.INTER_LINEAR)
        filename=PATH+'validation_masks/'+file
        cv2.imwrite(filename, mask)
    print('created validation images and masks')
    return None    

In [55]:
create_validation_images_and_masks()

created validation images and masks


In [5]:
create_training_images_and_masks(10000)

created randomized training images and masks
